# What it is

A script that takes an excel xlsx file containing the url to check and canoncial url pairings that a user wants to check. The tool outputs whether the canonical has been set at all, incorrectly, or correctly.

Ouputted file will contain the results of the check. Each tag checked will be on a seperate sheet labelled by the tag.

# Before Running All Cells

Check the the xlsx file containing canonicals that you want to check follows the correct format. To see the expected format, view the xlsx file "Example" under the Inputs folder. Then, place the xlsx file in the Check-Canonicals > Inputs folder.

Finally, enter file input below under [User Input](#user_input)


# How to Run
On top select Cell -> Run All

<a id='user_input'></a>

## User Input

Enter information below before running the cells.

In [ ]:

CANONICALS_WORKBOOK = 'Example.xlsx' # Name of xlsx containing canonicals you want to test
CANONICALS_WORKBOOK_SHEET = 'CanonicalsSheet' # Name of xlsx's sheet containing canonicals you want to test

# If an http(s) needs to be appended to the url, what should it be?
# https = True
# http = False
# This will only change things if the beginning of url path is not specified
IS_SSL = True

# Whether looking to test the review site or production site
# Used for Master Lock and SentrySafe
# Not be supported for other sites, leave False
IS_ML_REVIEW = False

## Import and Constant

Cells in this section import libraries, define where the ouputted file will go, and load the file the user wants to use to check canonicals.

In [ ]:
# Imports and constants

import pandas as pd
import xlrd
from xlutils.copy import copy
from datetime import datetime
import pandas as pd

import sys
sys.path.append('../')
import automatedtesting

CANONICALS_INPUT_FOLDER = 'Inputs/'
CANONICALS_OUTPUT_FOLDER = 'Results/'

CANONICALS_INPUT_WORKBOOK_PATH = CANONICALS_INPUT_FOLDER + CANONICALS_WORKBOOK

to_check = xlrd.open_workbook(CANONICALS_INPUT_WORKBOOK_PATH)
to_check_sheet = to_check.sheet_by_name(CANONICALS_WORKBOOK_SHEET)

check_wb = copy(to_check) 
check_sheet = check_wb.get_sheet(CANONICALS_WORKBOOK_SHEET)

OK = "OK"

## Functions

In this section, functions are defined to make the code easier to read and write tests for.

In [ ]:
def return_canonical_result(actual, expected, expected_status_code):
    if actual == "n/a":
        '''If canonical has not been set, set response to say that.'''
        match_result = "Canonical not set"

    elif actual == expected:
        if expected_status_code == 200:
            match_result = OK
        else:
            match_result = "Canonical is bad link, but is equal to expected. Consider changing canonical."
    else:
        match_result = 'Expected and actual canonicals do not match'
    return match_result

# Testing

The cells below are a check to make sure that the tool is working correctly. If one of these fails, and the canonical checker still runs, outputted file may be incorrect. Reach out or trouble shoot based on the outputted error.

When selecting 'Run All Cells', if one of these tests fails, the code will stop running at this cell. If you want to continue, you can select the 'Actual Check' cell and continue by running that, but it's highly advised against.

In [ ]:
# Test for checking that the canonical parser is working correctly.
# If this returns a warning, first check that the passed in url actually has the redirect.
def test_return_canonical(url, canonical):
    result = automatedtesting.return_canonical(url)
    if result == canonical:
        return True
    else:
        print("Error when parsing")
        return sys.exit(result)

test_return_canonical("https://www.masterlock.com/business-use/product/A1266NBLK",
                      "http://www.masterlock.com/business-use/product/A1266NBLK")   

## Actual Check

Now on to applying the logic.

In [ ]:
# These are what the headers of the outputted xlsx will be, along with the output printed after running this cell.
cols = ["URL Result", "URL", "URL Status Code", "Result", "Canonical Status Code",
        "Expected Canonical", "Actual Canonical"]

# This will be the ouputted table that will hold all of the results. It is currently empty to have a container
# to put results in.
list_of_results = pd.DataFrame(columns=cols)

#For every row in the input data, check to see that the canonical 1) exists 2) is what was desired
for i in range(1, len(check_sheet.rows)):
    
    # Clear variables before checking row
    actual_canonical = "n/a"
    match_result = "n/a"
    
    # Get data from the inputted file and clean the url input
    url_containing_canonical = automatedtesting.change_env(to_check_sheet.cell(i, 0).value, IS_ML_REVIEW, IS_SSL)
    expected_canonical = automatedtesting.change_env(to_check_sheet.cell(i, 1).value.strip(), IS_ML_REVIEW, IS_SSL)
    
    # Get status codes (200, 301, 404, etc) of the row's urls.
    url_status_code = automatedtesting.get_status_code(url_containing_canonical)
    canonical_status_code = automatedtesting.get_status_code(expected_canonical)
    
    # If the url is okay, the canonical url can then be checked
    # If not, return an Error result
    if url_status_code == 200:
        url_result = OK
        actual_canonical = automatedtesting.return_canonical(url_containing_canonical)
        match_result = return_canonical_result(actual_canonical, expected_canonical, canonical_status_code)
    else: 
        url_result = "Error when accessing url containing canonical. See status code."

    
    # Append the result to a dataframe for output later
    list_of_results.loc[i] = [url_result, url_containing_canonical, url_status_code, match_result,
                              canonical_status_code, expected_canonical, actual_canonical]

print(list_of_results)

## Create Result Output File

After running the cell below, the results gotten from checking canonicals will be placed in an xlsx with the current timestamp in the title and then outputted to the __Results__ folder.

In [ ]:
# Run to output the dataframe as an xlsx file in the 'Results' folder

OUTPUT_FILE = CANONICALS_OUTPUT_FOLDER + 'canonical-results_'+ datetime.now().strftime("%Y-%m-%d_%H-%M") + '.xlsx'

writer = pd.ExcelWriter(OUTPUT_FILE, engine='xlsxwriter',)
list_of_results.to_excel(writer, sheet_name='Canonicals', index=False)
writer.save()